In [8]:
import pandas as pd
import os
from datetime import date, timedelta
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pymysql

In [3]:
# 設定縣市名稱及列表、欄位名稱，先建立空的主表
city_dict = {
    "A":"新北市",
    "V":"臺北市",
    "C":"桃園市",
    "S":"臺中市",
    "U":"臺南市",
    "W":"高雄市"
}

file_date = (date.today() - timedelta(days=1)).strftime("%Y%m%d")
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\pet_regis"
file = f"20251110_pet_regis.csv"
file_path = os.path.join(folder, file)

df = pd.read_csv(file_path)

# 先還原欄位名
columns = [
    "area_id",
    "district",
    "登記單位數",
    "regis_count",
    "removal_count",
    "轉讓數",
    "變更數",
    "絕育數",
    "絕育除戶數",
    "免絕育數",
    "免絕育除戶數",
    "animal",
    "date",
    "city",
    "update_date"
]

df.columns = columns
df

,area_id,district,登記單位數,regis_count,removal_count,轉讓數,變更數,絕育數,絕育除戶數,免絕育數,免絕育除戶數,animal,date,city,update_date
0,207,207萬里區,0,0,0,0,0,0,0,0,0,0,2025/11/02,A,2025/11/13
1,208,208金山區,0,0,0,0,0,0,0,0,0,0,2025/11/02,A,2025/11/13
2,220,220板橋區,8,10,0,24,4,1,0,0,0,0,2025/11/02,A,2025/11/13
3,221,221汐止區,0,3,0,3,2,2,0,0,0,0,2025/11/02,A,2025/11/13
4,222,222深坑區,0,0,0,0,0,0,0,0,0,0,2025/11/02,A,2025/11/13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,847,847甲仙區,0,0,0,0,0,0,0,0,0,1,2025/11/10,W,2025/11/13
2624,848,848桃源區,0,0,0,0,0,0,0,0,0,1,2025/11/10,W,2025/11/13
2625,849,849那瑪夏區,0,0,0,0,0,0,0,0,0,1,2025/11/10,W,2025/11/13
2626,851,851茂林區,0,13,0,0,0,13,0,0,0,1,2025/11/10,W,2025/11/13


In [4]:
# 將city還原為中文名
df["city"] = df["city"].apply(lambda x: city_dict[x])

In [5]:
# 將不要的欄位去除
df.drop(columns=["area_id", "登記單位數", "轉讓數", "變更數",
             "絕育數", "絕育除戶數", "免絕育數", "免絕育除戶數"], axis=1, inplace=True)

# 將區的郵遞區號去除
df["district"] = df["district"].apply(lambda x: x[3:])

In [9]:
# join取得loc id
load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

conn = pymysql.connect(
    host=target_ip,
    port=target_port,
    user=username,
    password=password,
    database=db_name,
    charset='utf8mb4'
)

sql = "SELECT * FROM location"

df_loc = pd.read_sql(sql, conn)

df_loc

C:\Users\add41\AppData\Local\Temp\ipykernel_2832\1658383003.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_loc = pd.read_sql(sql, conn)


,loc_id,city,district,area,population
0,NTP001,新北市,板橋區,23.14,550256.0
1,NTP002,新北市,三重區,16.32,383035.0
2,NTP003,新北市,中和區,20.14,404507.0
3,NTP004,新北市,永和區,5.71,211631.0
4,NTP005,新北市,新莊區,19.74,421669.0
...,...,...,...,...,...
153,KSH034,高雄市,杉林區,104.00,10595.0
154,KSH035,高雄市,內門區,95.62,12609.0
155,KSH036,高雄市,茂林區,194.00,1838.0
156,KSH037,高雄市,桃源區,928.98,4137.0


In [10]:
df_loc = df_loc[["loc_id", "city", "district"]]

df = df.merge(df_loc, how="left", on=["city", "district"])

df.drop(columns=["city", "district"], axis=1, inplace=True)

In [11]:
# 將欄位重新排序
new_col = ["loc_id", "date", "animal",
           "regis_count", "removal_count", "update_date"]
df = df[new_col]

In [12]:
# 存檔至地端保存
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\pet_regis"
save_file = f"20251110_pet_regis_ETL.csv"
save_path = os.path.join(folder, save_file)

df.to_csv(save_path, index=False, encoding="utf-8")
print(f"{file_date}檔案儲存完畢！")

20251112檔案儲存完畢！


In [15]:
df["date"] = pd.to_datetime(df["date"])
df["update_date"] = pd.to_datetime(df["update_date"])
df

,loc_id,date,animal,regis_count,removal_count,update_date
0,NTP028,2025-11-02,0,0,0,2025-11-13
1,NTP027,2025-11-02,0,0,0,2025-11-13
2,NTP001,2025-11-02,0,10,0,2025-11-13
3,NTP011,2025-11-02,0,3,0,2025-11-13
4,NTP018,2025-11-02,0,0,0,2025-11-13
...,...,...,...,...,...,...
2623,KSH033,2025-11-10,1,0,0,2025-11-13
2624,KSH037,2025-11-10,1,0,0,2025-11-13
2625,KSH038,2025-11-10,1,0,0,2025-11-13
2626,KSH036,2025-11-10,1,13,0,2025-11-13


In [21]:
df["animal"] = df["animal"].astype(str)

df

,loc_id,date,animal,regis_count,removal_count,update_date
0,NTP028,2025-11-02,0,0,0,2025-11-13
1,NTP027,2025-11-02,0,0,0,2025-11-13
2,NTP001,2025-11-02,0,10,0,2025-11-13
3,NTP011,2025-11-02,0,3,0,2025-11-13
4,NTP018,2025-11-02,0,0,0,2025-11-13
...,...,...,...,...,...,...
2623,KSH033,2025-11-10,1,0,0,2025-11-13
2624,KSH037,2025-11-10,1,0,0,2025-11-13
2625,KSH038,2025-11-10,1,0,0,2025-11-13
2626,KSH036,2025-11-10,1,13,0,2025-11-13


In [22]:
# 更新至資料庫
sql = "INSERT INTO pet_regis (loc_id, date, animal, regis_count, removal_count, update_date)" \
        "VALUES(%s, %s, %s, %s, %s, %s)"

conn = pymysql.connect(
    host=target_ip,
    port=target_port,
    user=username,
    password=password,
    database=db_name,
    charset='utf8mb4'
)

cursor = conn.cursor()

data = list(df.itertuples(index=False, name=None))


try:
    cursor.executemany(sql, data)
    conn.commit()
    print("資料寫入資料庫成功！")
except Exception as e:
    print(f"資料寫入資料褲時發生錯誤：{e}")
    conn.rollback()
finally:
    cursor.close()
    conn.close()

資料寫入資料庫成功！
